# Setup

In [2]:
import yaml
import pandas as pd
import numpy as np

import clipboard

from names import *
from styles import *

In [3]:
parameter_name_map = {
    'loc': '$\mu$',
    'sigma_c': '$\sigma_c$',
    'sigma_s': '$\sigma_s$',
    'scale': '$\sigma$',
    'sigma': '$\sigma$',
    'h': '$h$',
    'density': 'Density',
    'intensity': 'Intensity',
    'size': 'Kernel size'
}

type_name_map = {
    'const': 'Constant',
    'samples_num': 'Interpolation',
    'samples_step': 'Increments'
}

In [4]:
with open('data/optim_config.yaml') as f:
    data = yaml.safe_load(f)

params = []
keys = ['start', 'stop', 'num', 'step']
for f, par_dict in data.items():
    for p, config in par_dict.items():
        t = config['type']
        s_dict = config['params']
        s_vals = [s_dict.get(k, 0) for k in keys]
        ns = {
            'Type': type_name_map[t],
            'Filter': filter_name_map[f],
            'Parameter': parameter_name_map[p],
        }
        ns.update(s_dict)
        params.append(ns)

df = pd.DataFrame(params)
df.sort_values(['Type', 'Filter', 'Parameter'], inplace=True)
df = df.drop(columns='val')
df = df[df['Type'] != 'Constant']

dfa = df[df['Type'] == 'Increments']
dfa.drop(columns=['Type', 'num'], inplace=True)
dfb = df[df['Type'] == 'Interpolation']
dfb.drop(columns=['Type', 'step'], inplace=True)

/Users/Anton/anaconda3/envs/iaml/lib/python3.7/site-packages/pandas/core/frame.py:4167: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


# Tables

## Parameter tables

In [17]:
levels_a = pd.MultiIndex.from_frame(dfa).to_frame()
levels_a = levels_a[[]]

In [8]:
dfa.to_csv('out/param_table_a.csv')

In [24]:
levels_a.to_latex(
    'out/param_table_a.tex',
    multicolumn=True, 
    multirow=True, 
    decimal='.', 
    index=True, 
    escape=False,
    #float_format='{:2.2f}',
    na_rep='NA')

In [18]:
levels_b = pd.MultiIndex.from_frame(dfb).to_frame()
levels_b = levels_b[[]]

In [19]:
dfb.to_csv('out/param_table_b.csv')

In [20]:
levels_b.to_latex(
    'out/param_table_b.tex',
    multicolumn=True, 
    multirow=True, 
    decimal='.', 
    index=True, 
    escape=False,
    #float_format='{:2.2f}',
    na_rep='NA')

## Iris param table

In [10]:
names = ['1x', '1.5x', '2x', '5x', '10x']
rows = []
for n in names:
    with open(f'data/iris/{n}.yaml') as f:
        d = yaml.safe_load(f)['filters']
        res = []
        for k, v in d.items():
            for (pk, pv) in v.items():
                res.append((filter_name_map[k], n, parameter_name_map[pk], pv))
        d = pd.DataFrame(res, columns=['Filter', 'Experiment', 'Parameter', 'Value'])
        #d = d.pivot(columns='experiment')
        
        rows.append(d)

frame = pd.concat(rows)

In [21]:
f = frame.pivot(columns='Experiment', index=['Filter', 'Parameter'])
f = f.reindex(names, level=1, axis=1)

In [13]:
pd.options.display.float_format = '${:,.4f}'.format

In [14]:
f.to_latex(
    'out/iris-params.tex',
    multicolumn=True, 
    multirow=True, 
    decimal='.', 
    index=True, 
    longtable=False,
    escape=False,
    float_format=lambda x: '{:0.4f}'.format(x),
    na_rep='NA')

In [15]:
f.to_csv('out/iris-params.csv')